In [1]:
import yfinance as yf
import pandas as pd
import numpy as np


In [2]:
ticker = "NVDA"
stock = yf.Ticker(ticker)

expirations = stock.options
expirations[:5]


('2026-01-02', '2026-01-09', '2026-01-16', '2026-01-23', '2026-01-30')

In [3]:
expiry = expirations[0]  # nearest expiry
chain = stock.option_chain(expiry)

calls = chain.calls
puts = chain.puts


In [4]:
def clean_chain(df):
    return df[[
        "strike",
        "lastPrice",
        "bid",
        "ask",
        "impliedVolatility",
        "inTheMoney",
        "openInterest"
    ]].dropna()

calls = clean_chain(calls)
puts = clean_chain(puts)


In [5]:
spot = stock.history(period="1d")["Close"].iloc[-1]

calls["dist"] = abs(calls["strike"] - spot)
puts["dist"] = abs(puts["strike"] - spot)

atm_call = calls.sort_values("dist").iloc[0]
atm_put = puts.sort_values("dist").iloc[0]


In [6]:
p_vol_expansion = 0.72  # example from Phase 2


In [7]:
def select_strategy(p):
    if p > 0.65:
        return "LONG_VOL"
    elif p < 0.35:
        return "SHORT_VOL"
    else:
        return "NEUTRAL"

strategy = select_strategy(p_vol_expansion)
strategy


'LONG_VOL'

In [13]:
if strategy == "LONG_VOL":
    trade = {
        "type": "ATM STRADDLE",
        "call_strike": atm_call["strike"],
        "put_strike": atm_put["strike"],
        "expiry": expiry,
        "rationale": "High probability IV expansion from causal shock"
    }

elif strategy == "SHORT_VOL":
    trade = {
        "type": "IRON CONDOR / CREDIT SPREAD",
        "expiry": expiry,
        "rationale": "Low probability of volatility expansion"
    }

else:
    trade = {
        "type": "CALENDAR or WAIT",
        "expiry": expiry,
        "rationale": "Volatility unclear, avoid theta traps"
    }


In [14]:

trade


{'type': 'ATM STRADDLE',
 'call_strike': np.float64(190.0),
 'put_strike': np.float64(190.0),
 'expiry': '2026-01-02',
 'rationale': 'High probability IV expansion from causal shock'}